# **Bitcoin price forecasting - GeneralizedLinearRegression**
### Big Data Computing final project - A.Y. 2022 - 2023
Prof. Gabriele Tolomei

MSc in Computer Science

La Sapienza, University of Rome

### Author
Corsi Danilo - corsi.1742375@studenti.uniroma1.it



# Dependencies, Libraries and Tools

In [1]:
JAVA_HOME = "/usr/lib/jvm/java-8-openjdk-amd64"
MODEL_NAME = "GeneralizedLinearRegression"
SLOW_OPERATION = False

In [2]:
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
#Install some useful dependencies
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from itertools import cycle

import plotly.express as px

import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
import gc

import pandas as pd
import numpy as np
import json

import matplotlib.pyplot as plt
import seaborn as sns

# !pip install -U -q PyDrive # To use files that are stored in Google Drive directly (e.g., without downloading them from an external URL)
# !apt install openjdk-8-jdk-headless -qq
# import os
# os.environ["JAVA_HOME"] = JAVA_HOME

In [4]:
# Install Spark and related dependencies
!pip install pyspark

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

from pyspark.ml.regression import LinearRegression, RandomForestRegressor, GBTRegressor
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation
from pyspark.ml import Pipeline

# Create the session
conf = SparkConf().\
                set('spark.ui.port', "4050").\
                set('spark.executor.memory', '4G').\
                set('spark.driver.memory', '45G').\
                set('spark.driver.maxResultSize', '10G').\
                set("spark.kryoserializer.buffer.max", "1G").\
                setAppName("BitcoinPriceForecasting").\
                setMaster("local[*]")

# Create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285388 sha256=858592ef7e46ae2a9884673f966e24b5c8bda44b6c72102c0e417003fd47df92
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


# Link to Google Drive

In [5]:
# Define GDrive paths
GDRIVE_DIR = "/content/drive"

GDRIVE_DATASET_OUTPUT_DIR = GDRIVE_DIR + "/MyDrive/BDC/project/datasets/output"

GDRIVE_DATASET_NAME = "bitcoin_blockchain_data_30min"
GDRIVE_DATASET_NAME_ENG = GDRIVE_DATASET_NAME + "_eng"

GDRIVE_DATASET_NAME_EXT_ENG  = "/" + GDRIVE_DATASET_NAME_ENG + ".parquet"

GDRIVE_DATASET_NAME_ENG = GDRIVE_DATASET_OUTPUT_DIR + GDRIVE_DATASET_NAME_EXT_ENG


In [6]:
# Point Colaboratory to our Google Drive
from google.colab import drive

drive.mount(GDRIVE_DIR, force_remount=True)

Mounted at /content/drive


In [7]:
# Load datasets into pyspark dataframe objects
df = spark.read.load(GDRIVE_DATASET_NAME_ENG,
                         format="parquet",
                         sep=",",
                         inferSchema="true",
                         header="true"
                    )

# Import my utilities

In [8]:
import sys
GDRIVE_UTILITIES_DIR = GDRIVE_DIR + "/MyDrive/BDC/project/utilities"
sys.path.append(GDRIVE_UTILITIES_DIR)

import shutil
shutil.rmtree(GDRIVE_UTILITIES_DIR + '/__pycache__')

import utilities

import importlib
importlib.reload(utilities)

<module 'utilities' from '/content/drive/MyDrive/BDC/project/utilities/utilities.py'>

# Evaluation of a simple model

In [9]:
GDRIVE_FEATURES_DIR = GDRIVE_DIR + "/MyDrive/BDC/project/features"

GDRIVE_ALL_FEATURES_NAME = "all_features"
GDRIVE_COR_MATRIX_FEATURES_NAME = "cm_features"

GDRIVE_ALL_FEATURES_NAME_EXT = "/" + GDRIVE_ALL_FEATURES_NAME + ".json"
GDRIVE_COR_MATRIX_FEATURES_NAME_EXT = "/" + GDRIVE_COR_MATRIX_FEATURES_NAME + ".json"

GDRIVE_ALL_FEATURES = GDRIVE_FEATURES_DIR + GDRIVE_ALL_FEATURES_NAME_EXT
GDRIVE_COR_MATRIX_FEATURES = GDRIVE_FEATURES_DIR + GDRIVE_COR_MATRIX_FEATURES_NAME_EXT

In [10]:
# Loading all features
with open(GDRIVE_ALL_FEATURES, "r") as f:
    all_features = json.load(f)
print(all_features)

# Loading correlation matrix features
with open(GDRIVE_COR_MATRIX_FEATURES, "r") as f:
    cm_features = json.load(f)
print(cm_features)

# Set the depended variable
TARGET_VAL = 'tomorrow-market-price'

# Set the features label
FEATURES_LABEL = "features"

['total-bitcoins', 'market-cap', 'trade-volume', 'blocks-size', 'avg-block-size', 'n-transactions-total', 'n-transactions-per-block', 'hash-rate', 'difficulty', 'miners-revenue', 'transaction-fees-usd', 'n-unique-addresses', 'n-transactions', 'estimated-transaction-volume-usd']
['market-cap', 'miners-revenue', 'estimated-transaction-volume-usd', 'n-transactions-total', 'blocks-size', 'total-bitcoins', 'difficulty']


In [11]:
# Valid performances with all the features
utilities.evaluate_simple_model(df, all_features, MODEL_NAME, FEATURES_LABEL, TARGET_VAL)

Output hidden; open in https://colab.research.google.com to view.

In [12]:
# Valid performances with the corr matrix features
utilities.evaluate_simple_model(df, cm_features, MODEL_NAME, FEATURES_LABEL, TARGET_VAL)

Output hidden; open in https://colab.research.google.com to view.

# Hyperparameter tuning w/ Time-series cross validation

In [13]:
# Split proportion list
PORTION_LIST = [0.6, 0.7, 0.8, 0.9]

In [14]:
# Generalized Linear Regression params
# params = {
#     'maxIter' : [5, 10, 50, 80], # max number of iterations (>=0), default:25
#     'regParam' : [0, 0.1, 0.2], # regularization parameter (>=0), default:0.0
#     'family': ['gaussian', 'gamma'], # The name of family which is a description of the error distribution to be used in the model.
#     'link': ['identity', 'inverse'] # which provides the relationship between the linear predictor and the mean of the distribution function.
# }

params = {
    'maxIter' : [5, 10, 50, 80], # max number of iterations (>=0), default:25
    'regParam' : [0, 0.1, 0.2], # regularization parameter (>=0), default:0.0
    'family': ['gaussian', 'gamma'], # The name of family which is a description of the error distribution to be used in the model.
    'link': ['identity', 'inverse'] # which provides the relationship between the linear predictor and the mean of the distribution function.
}

In [15]:
results = utilities.autoTuning(df, cm_features, params, PORTION_LIST, MODEL_NAME, FEATURES_LABEL, TARGET_VAL)
results

,Model,Proportion,Parameters,RMSE,MAPE,MAE,Variance,R2,Adjusted_R2,Time,Predictions
0,GeneralizedLinearRegression,0.9,"[5, 0, gaussian, identity]",430.836224,0.014122,287.836999,6.228715e+06,0.968863,0.968851,0.877942,"DataFrame[tomorrow-market-price: double, predi..."


In [17]:
# Generalized Linear Regression params
params = {
    'maxIter' : [5], # max number of iterations (>=0), default:25
    'regParam' : [0], # regularization parameter (>=0), default:0.0
    'family': ['gaussian'], # The name of family which is a description of the error distribution to be used in the model.
    'link': ['identity'] # which provides the relationship between the linear predictor and the mean of the distribution function.
}

In [16]:
## Cross Validation Parameter
# Multiple Splits Time Series Cross Validation
mul_cv = {'cv_type':'mulTs',
          'kSplits': 5}

# Blocked Time Series Cross Validation
blk_cv = {'cv_type':'blkTs',
          'kSplits': 10}

In [18]:
results_mul_cv, trained_models_mul_cv = utilities.tsCrossValidation(df, cm_features, params, mul_cv, MODEL_NAME, FEATURES_LABEL, TARGET_VAL)
results_mul_cv

,Model,CV_type,Splits,Train&Validation,Parameters,RMSE,MAPE,MAE,Variance,R2,Adjusted_R2,Time
0,GeneralizedLinearRegression,mulTs,1,"(21030, 21029)","[5, 0, gaussian, identity]",338.328884,0.022718,169.111114,1.957462e+07,0.993768,0.993767,0.596443
1,GeneralizedLinearRegression,mulTs,2,"(42059, 21029)","[5, 0, gaussian, identity]",727.038972,0.120319,679.764985,5.727091e+06,0.891042,0.891016,0.725964
2,GeneralizedLinearRegression,mulTs,3,"(63088, 21029)","[5, 0, gaussian, identity]",669.821163,0.071264,621.032877,2.882406e+06,0.803533,0.803487,0.768801
3,GeneralizedLinearRegression,mulTs,4,"(84117, 21029)","[5, 0, gaussian, identity]",2402.589061,0.045645,2055.730818,2.343816e+08,0.971661,0.971654,1.357202
4,GeneralizedLinearRegression,mulTs,5,"(105146, 21029)","[5, 0, gaussian, identity]",786.999088,0.026098,635.377354,9.265717e+07,0.993443,0.993441,1.420923


In [19]:
results_blk_cv, trained_models_blk_cv = utilities.tsCrossValidation(df, cm_features, params, blk_cv, MODEL_NAME, FEATURES_LABEL, TARGET_VAL)
results_blk_cv

,Model,CV_type,Splits,Train&Validation,Parameters,RMSE,MAPE,MAE,Variance,R2,Adjusted_R2,Time
0,GeneralizedLinearRegression,blkTs,1,"(10093, 2524)","[5, 0, gaussian, identity]",20.440124,0.031359,18.579107,9.264988e+02,0.170188,0.168540,0.532491
1,GeneralizedLinearRegression,blkTs,2,"(10093, 2524)","[5, 0, gaussian, identity]",56.790467,0.018705,36.436805,2.442916e+05,0.986326,0.986299,0.620330
2,GeneralizedLinearRegression,blkTs,3,"(10093, 2524)","[5, 0, gaussian, identity]",825.121973,0.072540,705.959905,3.856806e+06,0.867676,0.867413,0.550067
3,GeneralizedLinearRegression,blkTs,4,"(10093, 2524)","[5, 0, gaussian, identity]",141.380960,0.019697,125.617935,7.307110e+04,0.546363,0.545462,0.501481
4,GeneralizedLinearRegression,blkTs,5,"(10093, 2524)","[5, 0, gaussian, identity]",285.712596,0.021238,229.631391,9.699016e+05,0.910784,0.910607,0.513044
5,GeneralizedLinearRegression,blkTs,6,"(10093, 2524)","[5, 0, gaussian, identity]",356.692876,0.035043,240.088297,9.217483e+05,0.842944,0.842632,0.473088
6,GeneralizedLinearRegression,blkTs,7,"(10093, 2524)","[5, 0, gaussian, identity]",650.534860,0.015494,400.436602,4.797130e+07,0.991333,0.991315,0.487132
7,GeneralizedLinearRegression,blkTs,8,"(10093, 2524)","[5, 0, gaussian, identity]",887.298211,0.014912,689.115272,5.238469e+06,0.880310,0.880072,0.904038
8,GeneralizedLinearRegression,blkTs,9,"(10093, 2524)","[5, 0, gaussian, identity]",909.209995,0.023728,678.963174,2.392864e+07,0.964212,0.964141,0.830806
9,GeneralizedLinearRegression,blkTs,10,"(10093, 2524)","[5, 0, gaussian, identity]",1495.805437,0.049157,1121.344845,3.289154e+06,-1.321536,-1.326146,0.573650


# Comparison table

In [20]:
# Define what model_info and evaluators in the Model Comparison Table
model_info = ['Model','CV_type','Parameters']
evaluator_lst = ['RMSE','MAPE','MAE','Variance','R2','Adjusted_R2','Time']

# The the Cross Validation results would like to compare
comparison_lst = [results_mul_cv, results_blk_cv]

In [21]:
# Show the Comparison Table
pd.concat([utilities.modelComparison(cv_result, model_info, evaluator_lst) for cv_result in comparison_lst])

,Model,CV_type,Parameters,RMSE,MAPE,MAE,Variance,R2,Adjusted_R2,Time
0,GeneralizedLinearRegression,mulTs,"[5, 0, gaussian, identity]",984.955434,0.057209,832.203430,7.104457e+07,0.930689,0.930673,0.973867
0,GeneralizedLinearRegression,blkTs,"[5, 0, gaussian, identity]",562.898750,0.030188,424.617333,8.649431e+06,0.583860,0.583034,0.598613


# Training the final model

In [22]:
model = utilities.train_final_model(df, cm_features, params, MODEL_NAME, FEATURES_LABEL, TARGET_VAL)

In [23]:
GDRIVE_MODELS_DIR = GDRIVE_DIR + "/MyDrive/BDC/project/models"
GDRIVE_MODEL_NAME_EXT = GDRIVE_MODELS_DIR + "/" + MODEL_NAME

In [24]:
# Save the trained model
model.write().overwrite().save(GDRIVE_MODEL_NAME_EXT)